In [88]:
from keras.callbacks import LambdaCallback
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

# Text Generation

In this notebook we'll use deep learning to generate text in the style of an author.

In [69]:
#download dataset
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


corpus length: 600893


In [70]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char2int = dict((c, i) for i, c in enumerate(chars))
int2char = dict((i, c) for i, c in enumerate(chars))


total chars: 57


In [71]:
# cut the text into semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 200285


We use a sliding window to split the text.

In [72]:
sentences[:10]

['preface\n\n\nsupposing that truth is a woma',
 'face\n\n\nsupposing that truth is a woman--',
 'e\n\n\nsupposing that truth is a woman--wha',
 '\nsupposing that truth is a woman--what t',
 'pposing that truth is a woman--what then',
 'sing that truth is a woman--what then? i',
 'g that truth is a woman--what then? is t',
 'hat truth is a woman--what then? is ther',
 ' truth is a woman--what then? is there n',
 'uth is a woman--what then? is there not ']

In [73]:
next_chars[:10]

['n', 'w', 't', 'h', '?', 's', 'h', 'e', 'o', 'g']

Now lets convert the text into numbers

In [74]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char2int[char]] = 1
    y[i, char2int[next_chars[i]]] = 1


In [75]:
x.shape

(200285, 40, 57)

# Callbacks

Callbacks allow us monitor our model whilst it's training and respond to it, be that by saving weights, inspecting error or calling some custom funciton.

In [76]:
from keras.callbacks import LambdaCallback, ModelCheckpoint, TensorBoard

The first callback well use is one for saving the model when it improves.

In [77]:
filepath = 'text-gen.hdf5'
checkpoint = ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True, mode='max')

The second one we'll use is for Tensorboard so we can visulize the training process more easily, this is important because we need to be able to spot overfitting so we know when to stop training the model. After training we can inspect the tensorflow logs by running the bellow to launch the server.

```
tensorboard --logdir==path/to/log-directory

```

After you've lanuch visting http://localhost:6006/ to view tensorboard.



In [78]:
tensorboard = TensorBoard(log_dir='./logs' ,histogram_freq=0,  
          write_graph=True, write_images=True)

And finally one to print some generated text as the model trains so we can see how stupid the model it is. This part has two steps, first we need to define a sampling function. During training the model will learn a function that given a seqeunce of characters can predict the probality of the next character. However if we always pick the most likely character we end up having a model that always repeats itself. We need to add a degree of randomness to it, this is what the sample function does.

In [79]:
from keras.utils import to_categorical

In [80]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

The second part is to generate the text, we first feed a input sequence into the model. After we get it's prediction we tag that back onto the sequence and feed that sequence back in.

In [81]:
def onehot_text(text):
    input_text = to_categorical([ char2int[char] for char in text],len(chars))
    return np.expand_dims(input_text,0)

In [82]:
def generate_text(text,length=100,temp=0.5):
    
    
    input_text  = onehot_text(text) 
    
    for i in range(length):
        
        #predict next char
        char = int2char[sample(model.predict(input_text)[0],temp)]
        #add to text
        text += char
        #use the next 40 chars for input
        input_text = onehot_text(text[-40:])     
    
    return text

The on epoch end function will get called are each round of traning.

In [ ]:
def on_epoch_end(epoch,logs):
    
    
    print(f'--------- Epoch {epoch}----------\n\n')
    seed_text = np.random.choice(sentences)
    
    for temp in [0.2,0.5,1,1.2]:
        text = generate_text(seed_text,temp=temp)
        print(f"{text}\n\n")
    
#Have to pass to a lambda callback to use with keras
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# Model

I'm still experimenting with the model architecture, currently the results are still a little non-sensical.

In [99]:
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM,Dropout, TimeDistributed, Bidirectional
from keras.optimizers import RMSprop

In [101]:
model = Sequential()
#When stacking LSTM we need return_sequences=True
model.add(LSTM(128, input_shape=(maxlen, len(chars)),return_sequences=True ))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))


optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Training

Now we're ready to train our model!

In [102]:
h = model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[checkpoint,print_callback,tensorboard])

Epoch 1/20
200285/200285 [==============================] - 161s 802us/step - loss: 2.4636
--------- Epoch 0----------


/home/dom/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:435: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


of omission, an additional
seduction undersest and the some the prest of the and the som the are and for the and of the some and a the more 


of omission, an additional
seduction undice of the reess of the mens and flom and precurtuous and dect a concest of the the to he wist, in t


of omission, an additional
seduction undeess of the seal of awents have beakgrend
rests wetply this arely evervesmive yen wordh ol the cardi


of omission, an additional
seduction undeession of when ons
gribes,
geaine. ralal, do 
basam kean--will
precrests ordlates merepiast to cune


Epoch 2/20
200285/200285 [==============================] - 155s 772us/step - loss: 1.9429
--------- Epoch 1----------
ut
already it broadens and widens anew, the man and the stand in the conscience and strenger to the more the morality and stance the man and


ut
already it broadens and widens anew, the time who be the the so it the self and be a stance and perhopers of the conscient the less hid m


ut
already it broadens 

/home/dom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


longer blame, for it is irrational to
blers and the man not the self the an an and the self the conscience in a science to the to the sould 


longer blame, for it is irrational to
blecter of man any the appers to that a one and some where delief, is one
the are and that which the n


longer blame, for it is irrational to
bll eoss prongrement to phyare phent is his timm himself
for the anttiscary parscular itself estress s


longer blame, for it is irrational to
blak in at att the narpherence by so ma, desitel maounds is, and one mard., pain as that wimmity brt t


Epoch 4/20
200285/200285 [==============================] - 152s 761us/step - loss: 1.8206
--------- Epoch 3----------
personal considerations of advantage is the suster of the some of the self the some of the constion and the some in the man the self so the 


personal considerations of advantage is and the not moral, the dight of the interte presents of the notnated and siscelys to the force the i


personal considerations

KeyboardInterrupt: 